[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter9_seminar_notebook.ipynb)

---

# Chapter 9 Seminar: Prophet and TBATS - Practice

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Seminar Objectives

In this practical seminar, you will:
1. Identify multiple seasonal patterns in time series data
2. Apply TBATS for high-frequency data with multiple seasonalities
3. Build and customize Prophet models
4. Add holiday effects and external regressors to Prophet
5. Compare TBATS vs Prophet performance

## Setup

In [ ]:
# Install required packages (for Colab)
import sys
if 'google.colab' in sys.modules:
    !pip install prophet tbats -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Check for Prophet
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("Prophet not installed. Install with: pip install prophet")

# Check for TBATS
try:
    from tbats import TBATS, BATS
    HAS_TBATS = True
except ImportError:
    HAS_TBATS = False
    print("TBATS not installed. Install with: pip install tbats")

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666'}

print(f"Prophet available: {HAS_PROPHET}")
print(f"TBATS available: {HAS_TBATS}")
print("Setup complete!")

## Exercise 1: Generate Data with Multiple Seasonalities

**Task:** Create synthetic data with daily and weekly seasonal patterns to understand multiple seasonality.

In [ ]:
# Generate hourly data with multiple seasonalities
np.random.seed(42)

# 4 weeks of hourly data
n_hours = 24 * 7 * 4  # 672 hours (4 weeks)
t = np.arange(n_hours)

# Base level
base = 100

# Trend (slight increase)
trend = 0.01 * t

# Daily seasonality (period = 24 hours)
# Peak at noon (hour 12) and evening (hour 19)
hours = t % 24
daily = 10 * np.exp(-((hours - 12) ** 2) / 20) + 8 * np.exp(-((hours - 19) ** 2) / 15)

# Weekly seasonality (period = 168 hours)
# Lower on weekends (day 5-6, i.e., Sat-Sun)
day_of_week = (t // 24) % 7
weekly = np.where(day_of_week >= 5, -8, 0)  # Weekend effect

# Noise
noise = np.random.normal(0, 3, n_hours)

# Combined series
y = base + trend + daily + weekly + noise

# Create timestamps
start_date = pd.Timestamp('2024-01-01')
dates = pd.date_range(start=start_date, periods=n_hours, freq='H')

# Create DataFrame
df = pd.DataFrame({'ds': dates, 'y': y})

print(f"Generated {len(df)} observations")
print(f"Date range: {df['ds'].min()} to {df['ds'].max()}")
print(f"\nSeasonalities:")
print(f"  - Daily (s=24): Peak at noon and evening")
print(f"  - Weekly (s=168): Lower on weekends")

In [ ]:
# Visualize the components
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Full series
axes[0].plot(df['ds'], df['y'], color=COLORS['blue'], linewidth=0.5, alpha=0.7)
axes[0].set_title('Full Series: Multiple Seasonalities', fontweight='bold')
axes[0].set_ylabel('Value')

# One week detail
one_week = df.iloc[:168]  # First week
axes[1].plot(one_week['ds'], one_week['y'], color=COLORS['green'], linewidth=1)
axes[1].set_title('One Week: Daily Pattern Visible', fontweight='bold')
axes[1].set_ylabel('Value')

# Mark weekend
weekend_mask = (one_week['ds'].dt.dayofweek >= 5)
axes[1].fill_between(one_week['ds'], one_week['y'].min(), one_week['y'].max(),
                     where=weekend_mask, alpha=0.2, color='red', label='Weekend')
axes[1].legend(loc='upper right')

# Average daily pattern
df['hour'] = df['ds'].dt.hour
hourly_avg = df.groupby('hour')['y'].mean()
axes[2].bar(hourly_avg.index, hourly_avg.values, color=COLORS['orange'], alpha=0.7)
axes[2].set_title('Average Daily Pattern (Aggregated)', fontweight='bold')
axes[2].set_xlabel('Hour of Day')
axes[2].set_ylabel('Average Value')
axes[2].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.show()

print("\nNote: SARIMA with s=24 would miss the weekly pattern!")
print("We need TBATS or Prophet to capture both.")

## Exercise 2: TBATS Model

**Task:** Apply TBATS to handle multiple seasonal periods.

In [ ]:
if HAS_TBATS:
    # Split data
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]
    
    print(f"Training samples: {len(train)}")
    print(f"Test samples: {len(test)}")
    print(f"\nFitting TBATS with seasonal periods [24, 168]...")
    
    # Fit TBATS
    estimator = TBATS(seasonal_periods=[24, 168])
    tbats_model = estimator.fit(train['y'].values)
    
    print("\nTBATS Model Summary:")
    print(tbats_model.summary())
else:
    print("TBATS not available. Please install: pip install tbats")

In [ ]:
if HAS_TBATS:
    # Forecast
    forecast_tbats = tbats_model.forecast(steps=len(test))
    
    # Evaluate
    rmse_tbats = np.sqrt(mean_squared_error(test['y'], forecast_tbats))
    mae_tbats = mean_absolute_error(test['y'], forecast_tbats)
    mape_tbats = np.mean(np.abs((test['y'].values - forecast_tbats) / test['y'].values)) * 100
    
    print("TBATS Results")
    print("=" * 40)
    print(f"RMSE: {rmse_tbats:.2f}")
    print(f"MAE: {mae_tbats:.2f}")
    print(f"MAPE: {mape_tbats:.2f}%")

In [ ]:
if HAS_TBATS:
    # Plot TBATS forecast
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Full forecast
    axes[0].plot(train['ds'], train['y'], color=COLORS['blue'], label='Training', linewidth=0.5)
    axes[0].plot(test['ds'], test['y'], color=COLORS['green'], label='Actual', linewidth=1)
    axes[0].plot(test['ds'], forecast_tbats, color=COLORS['red'], label='TBATS Forecast', 
                 linewidth=1, linestyle='--')
    axes[0].axvline(x=test['ds'].iloc[0], color='black', linestyle='--', alpha=0.5)
    axes[0].set_title('TBATS: Full Forecast', fontweight='bold')
    axes[0].set_ylabel('Value')
    axes[0].legend(loc='upper left')
    
    # Zoomed view (first 3 days of test)
    n_zoom = 72  # 3 days
    axes[1].plot(test['ds'].iloc[:n_zoom], test['y'].iloc[:n_zoom], 
                 color=COLORS['green'], label='Actual', linewidth=1.5)
    axes[1].plot(test['ds'].iloc[:n_zoom], forecast_tbats[:n_zoom], 
                 color=COLORS['red'], label='TBATS Forecast', linewidth=1.5, linestyle='--')
    axes[1].set_title('TBATS: Zoomed View (First 3 Days)', fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Value')
    axes[1].legend(loc='upper left')
    
    plt.tight_layout()
    plt.show()

## Exercise 3: Prophet Basic Model

**Task:** Build a Prophet model with automatic seasonality detection.

In [ ]:
if HAS_PROPHET:
    # Prophet requires specific column names: 'ds' for datetime, 'y' for values
    train_prophet = train[['ds', 'y']].copy()
    test_prophet = test[['ds', 'y']].copy()
    
    print("Fitting Prophet model...")
    
    # Basic Prophet model
    prophet_model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=False,  # We don't have a full year
        changepoint_prior_scale=0.05,  # Default
        seasonality_mode='additive'
    )
    
    # Suppress verbose output
    prophet_model.fit(train_prophet)
    
    print("Prophet model fitted!")
else:
    print("Prophet not available. Please install: pip install prophet")

In [ ]:
if HAS_PROPHET:
    # Create future dataframe
    future = prophet_model.make_future_dataframe(periods=len(test), freq='H')
    print(f"Future dataframe shape: {future.shape}")
    
    # Predict
    forecast_prophet_df = prophet_model.predict(future)
    
    # Extract test period forecast
    forecast_prophet = forecast_prophet_df['yhat'].iloc[-len(test):].values
    
    # Evaluate
    rmse_prophet = np.sqrt(mean_squared_error(test['y'], forecast_prophet))
    mae_prophet = mean_absolute_error(test['y'], forecast_prophet)
    mape_prophet = np.mean(np.abs((test['y'].values - forecast_prophet) / test['y'].values)) * 100
    
    print("\nProphet Results")
    print("=" * 40)
    print(f"RMSE: {rmse_prophet:.2f}")
    print(f"MAE: {mae_prophet:.2f}")
    print(f"MAPE: {mape_prophet:.2f}%")

In [ ]:
if HAS_PROPHET:
    # Plot Prophet components
    fig = prophet_model.plot_components(forecast_prophet_df)
    plt.suptitle('Prophet: Decomposition of Components', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
if HAS_PROPHET:
    # Plot Prophet forecast
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Full forecast with uncertainty
    test_forecast = forecast_prophet_df.iloc[-len(test):]
    
    axes[0].plot(train['ds'], train['y'], color=COLORS['blue'], label='Training', linewidth=0.5)
    axes[0].plot(test['ds'], test['y'], color=COLORS['green'], label='Actual', linewidth=1)
    axes[0].plot(test_forecast['ds'], test_forecast['yhat'], color=COLORS['red'], 
                 label='Prophet Forecast', linewidth=1, linestyle='--')
    axes[0].fill_between(test_forecast['ds'], test_forecast['yhat_lower'], test_forecast['yhat_upper'],
                         alpha=0.2, color='red', label='95% Interval')
    axes[0].axvline(x=test['ds'].iloc[0], color='black', linestyle='--', alpha=0.5)
    axes[0].set_title('Prophet: Forecast with Uncertainty', fontweight='bold')
    axes[0].set_ylabel('Value')
    axes[0].legend(loc='upper left')
    
    # Zoomed view
    n_zoom = 72
    axes[1].plot(test['ds'].iloc[:n_zoom], test['y'].iloc[:n_zoom],
                 color=COLORS['green'], label='Actual', linewidth=1.5)
    axes[1].plot(test_forecast['ds'].iloc[:n_zoom], test_forecast['yhat'].iloc[:n_zoom],
                 color=COLORS['red'], label='Prophet Forecast', linewidth=1.5, linestyle='--')
    axes[1].fill_between(test_forecast['ds'].iloc[:n_zoom],
                         test_forecast['yhat_lower'].iloc[:n_zoom],
                         test_forecast['yhat_upper'].iloc[:n_zoom],
                         alpha=0.2, color='red')
    axes[1].set_title('Prophet: Zoomed View (First 3 Days)', fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Value')
    axes[1].legend(loc='upper left')
    
    plt.tight_layout()
    plt.show()

## Exercise 4: Prophet with Custom Seasonality

**Task:** Add custom monthly seasonality and tune Prophet parameters.

In [ ]:
if HAS_PROPHET:
    # Create a longer dataset for custom seasonality
    np.random.seed(123)
    n_days = 365  # One year of daily data
    
    dates_daily = pd.date_range(start='2024-01-01', periods=n_days, freq='D')
    
    # Components
    t = np.arange(n_days)
    trend = 100 + 0.05 * t
    weekly = 5 * np.sin(2 * np.pi * t / 7)  # Weekly seasonality
    monthly = 8 * np.sin(2 * np.pi * t / 30.5)  # Monthly seasonality
    yearly = 15 * np.sin(2 * np.pi * t / 365)  # Yearly seasonality
    noise = np.random.normal(0, 3, n_days)
    
    y_daily = trend + weekly + monthly + yearly + noise
    
    df_daily = pd.DataFrame({'ds': dates_daily, 'y': y_daily})
    
    # Split
    train_daily = df_daily.iloc[:-60]  # All but last 60 days
    test_daily = df_daily.iloc[-60:]  # Last 60 days
    
    print(f"Daily data: {len(df_daily)} observations")
    print(f"Train: {len(train_daily)}, Test: {len(test_daily)}")

In [ ]:
if HAS_PROPHET:
    # Prophet with custom seasonality
    prophet_custom = Prophet(
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        changepoint_prior_scale=0.1,  # More flexible trend
        seasonality_prior_scale=10,  # Default seasonality regularization
        seasonality_mode='additive'
    )
    
    # Add custom monthly seasonality
    prophet_custom.add_seasonality(
        name='monthly',
        period=30.5,
        fourier_order=5  # Number of Fourier terms
    )
    
    print("Fitting Prophet with custom monthly seasonality...")
    prophet_custom.fit(train_daily)
    
    # Predict
    future_daily = prophet_custom.make_future_dataframe(periods=60, freq='D')
    forecast_custom_df = prophet_custom.predict(future_daily)
    
    print("Model fitted!")

In [ ]:
if HAS_PROPHET:
    # Plot custom seasonality components
    fig = prophet_custom.plot_components(forecast_custom_df)
    plt.suptitle('Prophet: Custom Seasonality Components', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Evaluate
    forecast_custom = forecast_custom_df['yhat'].iloc[-60:].values
    
    rmse_custom = np.sqrt(mean_squared_error(test_daily['y'], forecast_custom))
    mae_custom = mean_absolute_error(test_daily['y'], forecast_custom)
    mape_custom = np.mean(np.abs((test_daily['y'].values - forecast_custom) / test_daily['y'].values)) * 100
    
    print("\nProphet with Custom Seasonality Results")
    print("=" * 45)
    print(f"RMSE: {rmse_custom:.2f}")
    print(f"MAE: {mae_custom:.2f}")
    print(f"MAPE: {mape_custom:.2f}%")

## Exercise 5: Prophet with Holiday Effects

**Task:** Add holiday effects to Prophet model.

In [ ]:
if HAS_PROPHET:
    # Define holidays
    holidays = pd.DataFrame({
        'holiday': 'major_holiday',
        'ds': pd.to_datetime([
            '2024-01-01',  # New Year
            '2024-01-06',  # Epiphany
            '2024-05-01',  # Labor Day
            '2024-12-01',  # Romania National Day
            '2024-12-25',  # Christmas
            '2024-12-26',  # Boxing Day
        ]),
        'lower_window': -1,  # Days before
        'upper_window': 1,   # Days after
    })
    
    print("Defined holidays:")
    print(holidays)
    
    # Create data with holiday effect
    y_with_holiday = y_daily.copy()
    
    # Add drop on holidays
    for holiday_date in holidays['ds']:
        idx = (dates_daily >= holiday_date - pd.Timedelta(days=1)) & \
              (dates_daily <= holiday_date + pd.Timedelta(days=1))
        y_with_holiday[idx] -= 15  # Drop of 15 units on holidays
    
    df_holiday = pd.DataFrame({'ds': dates_daily, 'y': y_with_holiday})
    train_holiday = df_holiday.iloc[:-60]
    test_holiday = df_holiday.iloc[-60:]

In [ ]:
if HAS_PROPHET:
    # Model WITHOUT holidays
    prophet_no_holiday = Prophet(
        weekly_seasonality=True,
        yearly_seasonality=True
    )
    prophet_no_holiday.fit(train_holiday)
    forecast_no_hol = prophet_no_holiday.predict(
        prophet_no_holiday.make_future_dataframe(periods=60, freq='D')
    )
    
    # Model WITH holidays
    prophet_with_holiday = Prophet(
        holidays=holidays,
        weekly_seasonality=True,
        yearly_seasonality=True,
        holidays_prior_scale=10  # Regularization for holiday effects
    )
    prophet_with_holiday.fit(train_holiday)
    forecast_with_hol = prophet_with_holiday.predict(
        prophet_with_holiday.make_future_dataframe(periods=60, freq='D')
    )
    
    print("Fitted models with and without holidays")

In [ ]:
if HAS_PROPHET:
    # Compare results
    pred_no_hol = forecast_no_hol['yhat'].iloc[-60:].values
    pred_with_hol = forecast_with_hol['yhat'].iloc[-60:].values
    
    rmse_no_hol = np.sqrt(mean_squared_error(test_holiday['y'], pred_no_hol))
    rmse_with_hol = np.sqrt(mean_squared_error(test_holiday['y'], pred_with_hol))
    
    print("Holiday Effects Comparison")
    print("=" * 45)
    print(f"{'Model':<25} {'RMSE':>10}")
    print("-" * 45)
    print(f"{'Without holidays':<25} {rmse_no_hol:>10.2f}")
    print(f"{'With holidays':<25} {rmse_with_hol:>10.2f}")
    print("-" * 45)
    print(f"Improvement: {(rmse_no_hol - rmse_with_hol)/rmse_no_hol * 100:.1f}%")

In [ ]:
if HAS_PROPHET:
    # Visualize holiday effect
    fig, ax = plt.subplots(figsize=(14, 5))
    
    # Show holiday effect component
    holiday_effect = forecast_with_hol[['ds', 'holidays']].set_index('ds')
    
    ax.bar(holiday_effect.index, holiday_effect['holidays'], width=1.5, 
           color=COLORS['red'], alpha=0.7)
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax.set_title('Prophet: Extracted Holiday Effect', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Holiday Effect')
    
    # Mark actual holidays
    for holiday_date in holidays['ds']:
        if holiday_date in holiday_effect.index:
            ax.axvline(x=holiday_date, color='green', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## Exercise 6: Model Comparison - TBATS vs Prophet

**Task:** Compare TBATS and Prophet on the same dataset.

In [ ]:
# Use the original hourly data for comparison
print("Model Comparison: TBATS vs Prophet")
print("=" * 55)

results = []

if HAS_TBATS:
    results.append(('TBATS', rmse_tbats, mae_tbats, mape_tbats))

if HAS_PROPHET:
    results.append(('Prophet', rmse_prophet, mae_prophet, mape_prophet))

if results:
    print(f"{'Model':<15} {'RMSE':>10} {'MAE':>10} {'MAPE (%)':>10}")
    print("-" * 55)
    for name, rmse, mae, mape in results:
        print(f"{name:<15} {rmse:>10.2f} {mae:>10.2f} {mape:>10.2f}")
    print("-" * 55)
    
    # Determine best
    best_idx = np.argmin([r[1] for r in results])  # By RMSE
    print(f"\nBest model (by RMSE): {results[best_idx][0]}")
else:
    print("No models available for comparison.")

In [ ]:
# Visual comparison
if HAS_TBATS and HAS_PROPHET:
    fig, ax = plt.subplots(figsize=(14, 5))
    
    n_plot = 72  # 3 days
    
    ax.plot(test['ds'].iloc[:n_plot], test['y'].iloc[:n_plot],
            color=COLORS['blue'], label='Actual', linewidth=2)
    ax.plot(test['ds'].iloc[:n_plot], forecast_tbats[:n_plot],
            color=COLORS['red'], label=f'TBATS (RMSE={rmse_tbats:.2f})', 
            linewidth=1.5, linestyle='--')
    ax.plot(test['ds'].iloc[:n_plot], forecast_prophet[:n_plot],
            color=COLORS['green'], label=f'Prophet (RMSE={rmse_prophet:.2f})',
            linewidth=1.5, linestyle=':')
    
    ax.set_title('TBATS vs Prophet: 3-Day Forecast Comparison', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

## Practice Problems

### Problem 1: Fourier Terms

For yearly seasonality with daily data (period=365), Prophet uses 10 Fourier terms by default.

**Question:** How many parameters does this add to the model?

In [ ]:
print("Problem 1 Solution")
print("=" * 50)

fourier_order = 10
print(f"Fourier order: {fourier_order}")
print(f"\nEach Fourier term adds 2 parameters:")
print(f"  - a_n for cos(2πnt/P)")
print(f"  - b_n for sin(2πnt/P)")
print(f"\nTotal parameters: {fourier_order} × 2 = {fourier_order * 2}")
print(f"\nFormula: s(t) = Σ[a_n cos(2πnt/P) + b_n sin(2πnt/P)]")

### Problem 2: Seasonality Mode

You observe that seasonal amplitude grows as the trend increases.

**Question:** Should you use additive or multiplicative seasonality?

In [ ]:
print("Problem 2 Solution")
print("=" * 50)

print("If seasonal amplitude GROWS with trend level:")
print("  → Use MULTIPLICATIVE seasonality")
print("")
print("Additive: Y = T + S + ε")
print("  → Seasonal amplitude is constant")
print("")
print("Multiplicative: Y = T × S × ε")
print("  → Seasonal amplitude scales with trend")
print("")
print("In Prophet: seasonality_mode='multiplicative'")

### Problem 3: Model Selection

You have:
- Hourly electricity demand data
- Daily, weekly, and yearly patterns
- Important holiday effects
- Temperature as external regressor

**Question:** TBATS or Prophet? Why?

In [ ]:
print("Problem 3 Solution")
print("=" * 50)

print("Answer: Prophet")
print("")
print("Reasons:")
print("  1. Holiday effects are important → Prophet has built-in support")
print("  2. External regressor (temperature) → Prophet supports this")
print("  3. Both handle multiple seasonalities")
print("")
print("TBATS limitations:")
print("  - No holiday effects")
print("  - No external regressors")
print("  - Would need separate preprocessing")
print("")
print("Prophet code:")
print("  model = Prophet(holidays=holidays_df)")
print("  model.add_regressor('temperature')")

### Problem 4: TBATS Interpretation

TBATS selects: Box-Cox λ=0.5, ARMA(1,0), with 3 harmonics for daily and 2 for weekly.

**Question:** What does λ=0.5 mean?

In [ ]:
print("Problem 4 Solution")
print("=" * 50)

print("Box-Cox transformation with λ = 0.5:")
print("")
print("  y^(λ) = (y^λ - 1) / λ")
print("")
print("  For λ = 0.5:")
print("  y^(0.5) = (√y - 1) / 0.5 = 2(√y - 1)")
print("")
print("This is approximately a SQUARE ROOT transformation!")
print("")
print("Interpretation:")
print("  - Stabilizes variance")
print("  - Variance was increasing with level")
print("  - Common values:")
print("    λ = 1: No transformation")
print("    λ = 0.5: Square root")
print("    λ = 0: Log transformation")

## Summary

### Key Takeaways

1. **Multiple Seasonalities**
   - Standard SARIMA handles only one seasonal period
   - TBATS and Prophet handle multiple periods automatically
   - Common: daily (24/7), weekly (7/168), yearly (365)

2. **TBATS**
   - T: Trigonometric seasonality (Fourier terms)
   - B: Box-Cox transformation (variance stabilization)
   - A: ARMA errors (autocorrelation)
   - T: Trend (level + slope)
   - S: Seasonal (multiple periods)
   - Best for: High-frequency, automatic selection, no external regressors

3. **Prophet**
   - Additive decomposition: y(t) = g(t) + s(t) + h(t) + ε
   - Automatic changepoint detection
   - Built-in holiday effects
   - External regressors supported
   - Best for: Business forecasting, interpretability, holidays important

4. **Model Selection**
   - TBATS: Technical applications, high-frequency
   - Prophet: Business applications, holidays, external factors

### Practical Workflow

1. Identify seasonal periods in your data
2. Check if holidays/external factors matter
3. Choose TBATS (automatic) or Prophet (interpretable)
4. Tune parameters (Fourier order, changepoint scale)
5. Compare with cross-validation
6. Always validate with proper time series CV!